In [1]:
import rasterio
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import pathlib
import mbtiles
import gdal

from modules.helpers import *
from modules.image_utils import ms_to_rgb, stretch_img

In [2]:
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
OUTPUT_DIR = 'data/toulon-laspezia-web'

In [3]:
ds = gdal.Open(OUTPUT_DIR + '/GE01_La_Spezia_2011_01_24_011651197010_0_MS.tif')
ds
kwargs = {'-co TILE_FORMAT': 'PNG'}
options = "-co TILE_FORMAT=PNG -co ZOOM_LEVEL_STRATEGY=UPPER"
#options = "-co TILE_FORMAT=PNG"
gdal.Translate(OUTPUT_DIR + '/test2.gpkg', ds, options=options)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002759BF99AE0> >

In [ ]:
pan_path = meta.loc['GE01_La_Spezia_2011_01_24_011651197010_0', 'pan_tif_path']
ms_path = meta.loc['GE01_La_Spezia_2011_01_24_011651197010_0', 'ms_tif_path']

In [ ]:
def trim_img(img, percentile):
    cutoff_top = np.percentile(img, 100 - percentile)
    cutoff_bottom = np.percentile(img, percentile)
    print(cutoff_bottom, cutoff_top)
    img[img > cutoff_top] = cutoff_top
    img[img < cutoff_bottom] = cutoff_bottom
    print(np.min(img), np.max(img))
    img = stretch_img(img, individual_bands=False)
    img *= 255
    print(np.min(img), np.max(img))
    return img

def geotiff_11_to_8(path_in, path_out, ms_or_pan, sensor='GE01'):
    if isinstance(path_in, str):
        path_in = pathlib.Path(path_in)
    if isinstance(path_in, str):
        path_out = pathlib.Path(path_out)
        
    with rasterio.open(path_in, 'r') as src:
        profile = src.profile
        profile['dtype'] = 'uint8'
        #print(profile)
        img = src.read()
        img = rasterio.plot.reshape_as_image(img)
        if ms_or_pan == 'ms':
            img = ms_to_rgb(img, sensor=sensor)
            profile['count'] = 3
        img = img / 2047
        img *= 255
        print(img.shape)
        img = trim_img(img, 0.5)
        print(img.shape)
        img = img.astype(np.uint8)
        img = rasterio.plot.reshape_as_raster(img)
        with rasterio.open(path_out, 'w', **profile) as dst:
            dst.write(img)
    return img

In [ ]:
pan_img = geotiff_11_to_8(
    path_in=pan_path, 
    path_out=OUTPUT_DIR + '/GE01_La_Spezia_2011_01_24_011651197010_0_PAN.tif',
    ms_or_pan='pan')
ms_img = geotiff_11_to_8(
    path_in=ms_path, 
    path_out=OUTPUT_DIR + '/GE01_La_Spezia_2011_01_24_011651197010_0_MS.tif', 
    ms_or_pan='ms', 
    sensor='GE01')

In [ ]:
plt.hist(ms_img.flatten(), bins=50)

In [ ]:
plt.hist(pan_img.flatten(), bins=50)

In [ ]:
px.histogram(pan_img.flatten())